In [ ]:
CREATE OR REPLACE TEMP VIEW transactions USING CSV
OPTIONS (path "/mnt/bronze/transactions.csv", header "true", inferSchema "true");

CREATE OR REPLACE TEMP VIEW products USING JSON
OPTIONS (path "/mnt/bronze/products.json");


In [ ]:
SELECT
  t.StoreID,
  p.ProductName,
  SUM(t.Quantity) AS TotalQuantity,
  DENSE_RANK() OVER (PARTITION BY t.StoreID ORDER BY SUM(t.Quantity) DESC) AS rank
FROM transactions t
JOIN products p ON t.ProductID = p.ProductID
GROUP BY t.StoreID, p.ProductName
HAVING rank <= 3;


In [ ]:
SELECT
  p.ProductName,
  SUM(t.Quantity) AS TotalQuantity
FROM transactions t
JOIN products p ON t.ProductID = p.ProductID
WHERE t.StoreID = 6378
GROUP BY p.ProductName
ORDER BY TotalQuantity DESC
LIMIT 1;


In [ ]:
SELECT
  StoreID,
  SUM(t.Quantity * p.Price) AS TotalRevenue,
  RANK() OVER (ORDER BY SUM(t.Quantity * p.Price) DESC) AS RevenueRank
FROM transactions t
JOIN products p ON t.ProductID = p.ProductID
GROUP BY StoreID
ORDER BY TotalRevenue DESC;
